# Current Work

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import os
import json

# notebook settings
pd.set_option('display.max_columns', None)

In [2]:
path = "./unsynced-data/crop-yield-data/"
crop_files = os.listdir(path)
crop_files = [f for f in crop_files if f.endswith('.csv')]
grid = gpd.read_file("./synced-data/empty-grid.geojson")


In [3]:
def reduce_simulations(df, string, grid_df):
    working_df = df[["Exp.", "Yield", "Biomass", "Duration", "lat", "long"]].groupby("Exp.").mean().reset_index()
    # clean up and turn df into geodataframe
    working_df.columns = map(str.lower, working_df.columns)
    working_df = working_df.rename(columns={"yield": "yield_" + string, "biomass": "biomass_" + string, "duration": "duration_" + string})
    working_df = gpd.GeoDataFrame(working_df, crs="EPSG:4326", geometry=gpd.points_from_xy(working_df['long'], working_df['lat']))
    working_df = working_df.drop(columns=["lat", "long", "exp."])
    # add working_df to grid_df where geometries intersect on geometry
    grid_df = gpd.sjoin(grid_df, working_df, how="left", op="intersects")
    grid_df = grid_df.drop(columns='index_right')
    grid_df.columns = map(str.lower, grid_df.columns)
    return grid_df

In [4]:
def reduce_models(df):
    # create a list of column names
    column_names = df.columns.tolist()
    column_names.remove("geometry")
    models = ["_ipsl", "_mpi", "_gfdl", "_mri"]
    
    for m in models:
        # remove  m from the column names
        column_names = [c.replace(m, "") for c in column_names]
    # remove duplicates in column_names list by converting to set and back to list
    column_names = list(set(column_names))
    # average all columns that match a column name
    for c in column_names:
        df[c] = df.filter(regex=c).mean(axis=1)
    # drop all columns that include a string from models
    df = df.drop(columns=[c for c in df.columns if any(m in c for m in models)])
    df = df.round(3)
    return df


In [5]:
# It feels like there should be a better way to remove/not write NAN's to a geojson file to save space

def geojson_no_nans(df, path):
    # First write a geopandas dataframe to a geojson file
    df.to_file(path, driver='GeoJSON')

    # Load the GeoJSON data from a file
    with open(path, 'r') as geojson_file:
        geojson_data = json.load(geojson_file)

    def remove_null_properties(feature):
        non_null_properties = {key: value for key, value in feature['properties'].items() if value is not None}
        feature['properties'] = non_null_properties

    # Loop through features and remove null properties
    for feature in geojson_data['features']:
        remove_null_properties(feature)

    # Save the modified GeoJSON to a new file
    with open(path, 'w') as output_file:
        json.dump(geojson_data, output_file, indent=2)

In [6]:
# a dictionary of column quanitiles is useful for styling the map in the frontend
def generate_quantiles(df):
    col_list = df.columns.tolist()
    col_list.remove("geometry")
    q_list = ["min", 0.01, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98, 0.99, "max"]
    working_dict = {}
    for col in col_list:
        quantile_dict = {}
        for quantile in q_list:
            if quantile == "min":
                quantile_value = df[col].min()
                quantile_dict[quantile] = quantile_value

            elif quantile == "max":
                quantile_value = df[col].max()
                quantile_dict[quantile] = quantile_value
            else:
                quantile_value = df[col].quantile(quantile)
                quantile_dict[f"quantile_{int(quantile * 100)}"] = quantile_value  
        working_dict[col] = quantile_dict
    return working_dict

In [7]:
# i want to copy the generate_quantiles function, but want to make it the ratio of the ssp126/historical columns and the ssp360/historical columns
def generate_ratio_quantiles(df):
    # col_list = all columns that include ssp126 or ssp360
    col_list = [c for c in df.columns if "ssp126" in c or "ssp370" in c]
    col_list = [c for c in col_list if "yield" in c]
    # yield_sorghum_future_ssp370
    q_list = ["min", 0.01, 0.02, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.98, 0.99, "max"]
    working_dict = {}
    for col in col_list:
        quantile_dict = {}
        crop = col.split("_")[1]
        # divide the column by the "yield_"+crop+"_future_ssp370"column
        df["working_col"] = df[col] / df["yield_"+crop+"_historical"]
        for quantile in q_list:
            if quantile == "min":
                quantile_value = df["working_col"].min()
                quantile_dict[quantile] = quantile_value
                if quantile_value == np.inf:
                    quantile_value = "none"
            elif quantile == "max":
                quantile_value = df["working_col"].max()
                if quantile_value == np.inf:
                    quantile_value = "none"
                quantile_dict[quantile] = quantile_value
            else:
                quantile_value = df["working_col"].quantile(quantile)
                quantile_dict[f"quantile_{int(quantile * 100)}"] = quantile_value  
                if quantile_value == "NaN":
                    quantile_value = "none"
        working_dict[col] = quantile_dict
    return working_dict

In [8]:
for c in crop_files:
    file_df = pd.read_csv(path + c)
    column_string = c.split("-")[0]
    grid = reduce_simulations(file_df, column_string, grid)
grid = grid.round(3)


/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3505: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/IPython/core/inter

In [9]:
# correcting column names removing the reference to a simulation in the historical column names
for c in grid.columns:
    if "historical" in c:
        grid[c.replace("_ssp370", "")] = grid[c]
        grid = grid.drop(columns=c)


In [10]:
grid

,geometry,yield_yams_future_ssp126_mri,biomass_yams_future_ssp126_mri,duration_yams_future_ssp126_mri,yield_groundnut_future_ssp370_ipsl,biomass_groundnut_future_ssp370_ipsl,duration_groundnut_future_ssp370_ipsl,yield_pigeonpea_future_ssp370_mri,biomass_pigeonpea_future_ssp370_mri,duration_pigeonpea_future_ssp370_mri,yield_yams_future_ssp370_ipsl,biomass_yams_future_ssp370_ipsl,duration_yams_future_ssp370_ipsl,yield_okra_baseline_era_5_africa.csv,biomass_okra_baseline_era_5_africa.csv,duration_okra_baseline_era_5_africa.csv,yield_yams_future_ssp370_gfdl,biomass_yams_future_ssp370_gfdl,duration_yams_future_ssp370_gfdl,yield_pigeonpea_future_ssp370_gfdl,biomass_pigeonpea_future_ssp370_gfdl,duration_pigeonpea_future_ssp370_gfdl,yield_soybean_future_ssp126_mri,biomass_soybean_future_ssp126_mri,duration_soybean_future_ssp126_mri,yield_bambaragroundnut_future_ssp126_mpi,biomass_bambaragroundnut_future_ssp126_mpi,duration_bambaragroundnut_future_ssp126_mpi,yield_okra_future_ssp370_mri,biomass_okra_future_ssp370_mri,duration_okra_future_ssp370_mri,yield_groundnut_future_ssp126_mri,biomass_groundnut_future_ssp126_mri,duration_groundnut_future_ssp126_mri,yield_pigeonpea_baseline_era_5_africa.csv,biomass_pigeonpea_baseline_era_5_africa.csv,duration_pigeonpea_baseline_era_5_africa.csv,yield_pigeonpea_future_ssp126_mpi,biomass_pigeonpea_future_ssp126_mpi,duration_pigeonpea_future_ssp126_mpi,yield_yams_future_ssp126_mpi,biomass_yams_future_ssp126_mpi,duration_yams_future_ssp126_mpi,yield_okra_future_ssp126_mpi,biomass_okra_future_ssp126_mpi,duration_okra_future_ssp126_mpi,yield_bambaragroundnut_future_ssp370_mri,biomass_bambaragroundnut_future_ssp370_mri,duration_bambaragroundnut_future_ssp370_mri,yield_pigeonpea_future_ssp370_ipsl,biomass_pigeonpea_future_ssp370_ipsl,duration_pigeonpea_future_ssp370_ipsl,yield_bambaragroundnut_future_ssp126_ipsl,biomass_bambaragroundnut_future_ssp126_ipsl,duration_bambaragroundnut_future_ssp126_ipsl,yield_bambaragroundnut_future_ssp370_mpi,biomass_bambaragroundnut_future_ssp370_mpi,duration_bambaragroundnut_future_ssp370_mpi,yield_pigeonpea_future_ssp370_mpi,biomass_pigeonpea_future_ssp370_mpi,duration_pigeonpea_future_ssp370_mpi,yield_yams_future_ssp370_mpi,biomass_yams_future_ssp370_mpi,duration_yams_future_ssp370_mpi,yield_okra_future_ssp370_ipsl,biomass_okra_future_ssp370_ipsl,duration_okra_future_ssp370_ipsl,yield_okra_future_ssp370_mpi,biomass_okra_future_ssp370_mpi,duration_okra_future_ssp370_mpi,yield_pigeonpea_future_ssp126_gfdl,biomass_pigeonpea_future_ssp126_gfdl,duration_pigeonpea_future_ssp126_gfdl,yield_yams_future_ssp126_gfdl,biomass_yams_future_ssp126_gfdl,duration_yams_future_ssp126_gfdl,yield_groundnut_baseline_era_5_africa.csv,biomass_groundnut_baseline_era_5_africa.csv,duration_groundnut_baseline_era_5_africa.csv,yield_soybean_future_ssp126_ipsl,biomass_soybean_future_ssp126_ipsl,duration_soybean_future_ssp126_ipsl,yield_okra_future_ssp126_gfdl,biomass_okra_future_ssp126_gfdl,duration_okra_future_ssp126_gfdl,yield_pigeonpea_future_ssp126_ipsl,biomass_pigeonpea_future_ssp126_ipsl,duration_pigeonpea_future_ssp126_ipsl,yield_groundnut_future_ssp370_gfdl,biomass_groundnut_future_ssp370_gfdl,duration_groundnut_future_ssp370_gfdl,yield_bambaragroundnut_future_ssp370_ipsl,biomass_bambaragroundnut_future_ssp370_ipsl,duration_bambaragroundnut_future_ssp370_ipsl,yield_groundnut_future_ssp370_mpi,biomass_groundnut_future_ssp370_mpi,duration_groundnut_future_ssp370_mpi,yield_okra_future_ssp126_ipsl,biomass_okra_future_ssp126_ipsl,duration_okra_future_ssp126_ipsl,yield_soybean_baseline_era_5_africa.csv,biomass_soybean_baseline_era_5_africa.csv,duration_soybean_baseline_era_5_africa.csv,yield_bambaragroundnut_future_ssp126_gfdl,biomass_bambaragroundnut_future_ssp126_gfdl,duration_bambaragroundnut_future_ssp126_gfdl,yield_soybean_future_ssp370_gfdl,biomass_soybean_future_ssp370_gfdl,duration_soybean_future_ssp370_gfdl,yield_soybean_future_ssp370_ipsl,biomass_soybean_future_ssp370_ipsl,duration_soybean

In [11]:
geojson_no_nans(grid, "./unsynced-data/summarized-crop-yield-data/crop-yields-all-models.geojson")
grid.to_csv("./unsynced-data/summarized-crop-yield-data/crop-yields-all-models.csv", index=False) 

In [12]:
grid_reduced = reduce_models(grid)

/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
/opt/homebrew/Caskroom/miniforge/base/envs/geo/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  T

In [13]:
grid_reduced

,geometry,yield_okra_baseline_era_5_africa.csv,biomass_okra_baseline_era_5_africa.csv,duration_okra_baseline_era_5_africa.csv,yield_pigeonpea_baseline_era_5_africa.csv,biomass_pigeonpea_baseline_era_5_africa.csv,duration_pigeonpea_baseline_era_5_africa.csv,yield_groundnut_baseline_era_5_africa.csv,biomass_groundnut_baseline_era_5_africa.csv,duration_groundnut_baseline_era_5_africa.csv,yield_soybean_baseline_era_5_africa.csv,biomass_soybean_baseline_era_5_africa.csv,duration_soybean_baseline_era_5_africa.csv,yield_bambaragroundnut_baseline_era_5_africa.csv,biomass_bambaragroundnut_baseline_era_5_africa.csv,duration_bambaragroundnut_baseline_era_5_africa.csv,yield_yams_baseline_era_5_africa.csv,biomass_yams_baseline_era_5_africa.csv,duration_yams_baseline_era_5_africa.csv,duration_groundnut_future_ssp370,duration_soybean_future_ssp126,biomass_bambaragroundnut_future_ssp126,duration_okra_historical,yield_yams_future_ssp370,yield_bambaragroundnut_future_ssp370,yield_soybean_historical,biomass_okra_historical,biomass_groundnut_future_ssp370,yield_bambaragroundnut_historical,biomass_pigeonpea_future_ssp126,biomass_groundnut_historical,biomass_soybean_historical,yield_groundnut_future_ssp126,biomass_bambaragroundnut_historical,duration_pigeonpea_future_ssp370,yield_yams_future_ssp126,biomass_okra_future_ssp370,yield_pigeonpea_future_ssp370,yield_bambaragroundnut_future_ssp126,duration_bambaragroundnut_future_ssp370,yield_groundnut_future_ssp370,biomass_soybean_future_ssp126,yield_soybean_future_ssp370,duration_yams_future_ssp126,yield_okra_future_ssp126,yield_pigeonpea_historical,yield_okra_historical,duration_soybean_historical,yield_okra_future_ssp370,biomass_yams_future_ssp126,duration_groundnut_historical,duration_pigeonpea_historical,duration_okra_future_ssp126,yield_soybean_future_ssp126,biomass_yams_future_ssp370,biomass_groundnut_future_ssp126,duration_pigeonpea_future_ssp126,duration_soybean_future_ssp370,duration_groundnut_future_ssp126,biomass_okra_future_ssp126,yield_groundnut_historical,duration_yams_historical,duration_bambaragroundnut_historical,biomass_pigeonpea_future_ssp370,biomass_pigeonpea_historical,duration_yams_future_ssp370,duration_okra_future_ssp370,biomass_bambaragroundnut_future_ssp370,yield_yams_historical,biomass_yams_historical,duration_bambaragroundnut_future_ssp126,biomass_soybean_future_ssp370,yield_pigeonpea_future_ssp126
0,POINT (19.25000 -34.25000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POINT (19.75000 -34.25000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POINT (20.25000 -34.25000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POINT (20.75000 -34.25000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POINT (21.25000 -34.25000),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...

In [14]:
geojson_no_nans(grid_reduced, "./synced-data/crop-yields-mean-models.geojson")
grid_reduced.to_csv("./synced-data/crop-yields-mean-models.csv", index=False)

In [15]:
geojson_no_nans(grid_reduced, "./unsynced-data/summarized-crop-yield-data/crop-yields-mean-models.geojson")
grid_reduced.to_csv("./unsynced-data/summarized-crop-yield-data/crop-yields-mean-models.csv", index=False)

In [16]:
quantiles = generate_quantiles(grid_reduced)
with open('./synced-data/crop-yield-data-quantiles.json', 'w') as fp:
    json.dump(quantiles, fp)
quantiles

{'yield_okra_baseline_era_5_africa.csv': {'min': 2750.947,
  'quantile_1': 3966.08572,
  'quantile_2': 4207.09428,
  'quantile_10': 6644.641200000001,
  'quantile_20': 8213.2318,
  'quantile_30': 8952.6333,
  'quantile_40': 9440.2256,
  'quantile_50': 9720.525,
  'quantile_60': 10004.11,
  'quantile_70': 10252.1214,
  'quantile_80': 10541.8528,
  'quantile_90': 11063.2052,
  'quantile_98': 11906.371560000003,
  'quantile_99': 12348.5315,
  'max': 13127.1},
 'biomass_okra_baseline_era_5_africa.csv': {'min': 4912.316,
  'quantile_1': 7082.44623,
  'quantile_2': 7512.63508,
  'quantile_10': 11865.3227,
  'quantile_20': 14666.4212,
  'quantile_30': 15987.067799999999,
  'quantile_40': 16857.4784,
  'quantile_50': 17357.949999999997,
  'quantile_60': 17864.44,
  'quantile_70': 18307.321399999997,
  'quantile_80': 18824.8632,
  'quantile_90': 19755.7157,
  'quantile_98': 21261.403440000006,
  'quantile_99': 22050.987499999996,
  'max': 23441.0},
 'duration_okra_baseline_era_5_africa.csv': {'

In [17]:
ratio_quantiles = generate_ratio_quantiles(grid_reduced)
with open('./synced-data/crop-yield-data-ratio-quantiles.json', 'w') as fp:
    json.dump(ratio_quantiles, fp)
ratio_quantiles

{'yield_yams_future_ssp370': {'min': 0.0932214041424385,
  'quantile_1': 0.17238240254670123,
  'quantile_2': 0.20148957998992886,
  'quantile_10': 0.46389880235647796,
  'quantile_20': 0.645757329119008,
  'quantile_30': 0.8783592310080179,
  'quantile_40': 0.9917399886863135,
  'quantile_50': 1.0444989804952596,
  'quantile_60': 1.0959856103977865,
  'quantile_70': 1.2026387174428843,
  'quantile_80': 1.240031899832002,
  'quantile_90': 1.3015083360792885,
  'quantile_98': 1.5148715087772098,
  'quantile_99': 1.648043543604546,
  'max': 7.693024278210249},
 'yield_bambaragroundnut_future_ssp370': {'min': 0.18390470656595,
  'quantile_1': 0.36454996910896814,
  'quantile_2': 0.38543664778987907,
  'quantile_10': 0.47053114333018686,
  'quantile_20': 0.6079618989721989,
  'quantile_30': 0.8091023166220712,
  'quantile_40': 0.9367917307586601,
  'quantile_50': 1.0358392186859415,
  'quantile_60': 1.0814486777575507,
  'quantile_70': 1.23922281649263,
  'quantile_80': 1.3538942576830006,

After running this notebook follow these steps to process the data for the app:
1. put unprocessed .csv into public/data-raw
2. run node scripts/format-yields-data.js (there are source and target filenames in this script if that ever changes)
3. run node scripts/generate-minimaps.js (update the minimap svg images with new data)